<a href="https://colab.research.google.com/github/liangliang6v6/GraphCondensation/blob/main/Homework6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Task 4(100 points):
Implement the game of tic-tac-toe (write a class that implements an agent
playing Tic Tac Toe and learning its Q function) using the Q-learning technique (see the
resources/links provided in class for more details). Clearly describe your evaluation metric and
demonstrate a few runs.

## Answer
Tic-Tac-Toe can be implemented as a reinforcement learning problem using Q-learning, where an agent learns an optimal policy by interacting with the game environment. The board is represented as a state space, with each state corresponding to a unique board configuration. The action space consists of available moves (empty positions). The agent follows an ε-greedy policy, where it explores random moves occasionally but primarily selects moves based on learned Q-values. The Q-function is updated iteratively using the Bellman equation, where the agent receives rewards for winning (+1), losing (-1), drawing (+0.5), or making a neutral move (0). Over thousands of training episodes, the agent gradually improves, shifting from random play to an optimized strategy.

The performance of the trained agent is evaluated based on its win rate against a random opponent. The evaluation metric includes win percentage, draw percentage, and loss percentage over multiple test games. Demonstrations of the runs involve playing against a human or a predefined opponent, where the AI chooses moves based on its learned Q-values. After training, the AI should consistently outperform a random player by making optimal decisions. For further details, Sutton and Barto’s Reinforcement Learning: An Introduction provides foundational insights into Q-learning. Additionally, OpenAI Gym and other online resources offer tutorials on implementing RL-based game agents.

In [3]:
import numpy as np
import random
import pickle

class TicTacToeQLearning:
    def __init__(self, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.q_table = {}
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.state = [' '] * 9
        self.player = 'X'

    def get_state(self):
        return ''.join(self.state)

    def available_actions(self):
        return [i for i in range(9) if self.state[i] == ' ']

    def make_move(self, action):
        if self.state[action] == ' ':
            self.state[action] = self.player
            return True
        return False

    # check status for win
    def check_winner(self):
        win_conditions = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
        for (i, j, k) in win_conditions:
            if self.state[i] == self.state[j] == self.state[k] and self.state[i] != ' ':
                return self.state[i]
        if ' ' not in self.state:
            return 'Draw'
        return None

    def reset(self):
        self.state = [' '] * 9

    def choose_action(self):
        state = self.get_state()
        if random.uniform(0, 1) < self.epsilon or state not in self.q_table:
            return random.choice(self.available_actions())  # Explore
        else:
            return max(self.q_table[state], key=self.q_table[state].get)  # Exploit

    def update_q_table(self, state, action, reward, next_state):
        if state not in self.q_table:
            self.q_table[state] = {a: 0 for a in range(9)}
        if next_state not in self.q_table:
            self.q_table[next_state] = {a: 0 for a in range(9)}

        old_q = self.q_table[state][action]
        max_future_q = max(self.q_table[next_state].values())
        self.q_table[state][action] = old_q + self.alpha * (reward + self.gamma * max_future_q - old_q)

    def train(self, episodes=10000):
        for i in range(episodes):
            if i % (episodes // 10) == 0:
                print(f"Training Progress: {i}/{episodes} episodes completed.")
            self.reset()
            state = self.get_state()
            while True:
                action = self.choose_action()
                self.make_move(action)
                next_state = self.get_state()
                winner = self.check_winner()

                if winner == 'X':
                    reward = 1
                elif winner == 'O':
                    reward = -1
                elif winner == 'Draw':
                    reward = 0.5
                else:
                    reward = 0

                self.update_q_table(state, action, reward, next_state)
                state = next_state

                if winner:
                    break


    def play_against_human(self):
        self.reset()
        while True:
            print("\n".join([" ".join(self.state[i:i+3]) for i in range(0, 9, 3)]))
            if self.player == 'X':
                action = self.choose_action()
                self.make_move(action)
                print(f"AI chooses position {action}")
            else:
                action = int(input("Enter your move (0-8): "))
                if self.state[action] != ' ':
                    print("Invalid move, try again.")
                    continue
                self.make_move(action)

            winner = self.check_winner()
            if winner:
                print("\n".join([" ".join(self.state[i:i+3]) for i in range(0, 9, 3)]))
                if winner == 'Draw':
                    print("It's a draw!")
                else:
                    print(f"{winner} wins!")
                break

            self.player = 'O' if self.player == 'X' else 'X'

# train and test the agent
ttt_agent = TicTacToeQLearning()
ttt_agent.train(50000)

print("Trained AI, now playing against human:")
ttt_agent.play_against_human()


Training Progress: 0/50000 episodes completed.
Training Progress: 5000/50000 episodes completed.
Training Progress: 10000/50000 episodes completed.
Training Progress: 15000/50000 episodes completed.
Training Progress: 20000/50000 episodes completed.
Training Progress: 25000/50000 episodes completed.
Training Progress: 30000/50000 episodes completed.
Training Progress: 35000/50000 episodes completed.
Training Progress: 40000/50000 episodes completed.
Training Progress: 45000/50000 episodes completed.
Trained AI, now playing against human:
     
     
     
AI chooses position 0
X    
     
     
Enter your move (0-8): 2
X   O
     
     
AI chooses position 4
X   O
  X  
     
Enter your move (0-8): 5
X   O
  X O
     
AI chooses position 7
X   O
  X O
  X  
Enter your move (0-8): 1
X O O
  X O
  X  
AI chooses position 3
X O O
X X O
  X  
Enter your move (0-8): 8
X O O
X X O
  X O
O wins!
